In [ ]:
!pip install ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.8/89.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 117.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
# =====================[ 0) PREP & INSTALL ]=====================
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["OMP_NUM_THREADS"] = "1"  # stabilitas di Colab

# Hindari konflik NumPy ↔ OpenCV: pakai NumPy 2.x bawaan, pin OpenCV headless yang cocok
%pip -q install --no-cache-dir \
  ultralytics==8.3.206 \
  roboflow \
  gdown \
  pyyaml \
  opencv-python-headless==4.10.0.84

# =====================[ 1) IMPORTS & UTILS ]====================
import sys, shutil, random, yaml, gc
from pathlib import Path
import numpy as np, cv2, torch

print(f"Python  : {sys.version.split()[0]}")
print(f"PyTorch : {torch.__version__} | CUDA: {torch.version.cuda} | GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"NumPy   : {np.__version__}")
print(f"OpenCV  : {cv2.__version__}")

random.seed(0); np.random.seed(0); torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cuda.matmul.allow_tf32 = True
cv2.setNumThreads(0)

# =====================[ 2) DOWNLOAD / LOAD DATASET ]============
from roboflow import Roboflow

ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY", "y3yC8pmucXxpgOpseeNn")
WORKSPACE = "concrete-column-crack-detector"
PROJECT   = "concrete-column-crack-detection-nckx9"
VERSION   = 6
EXPORT    = "yolov12"  # detection

DATA_DIR = Path("/content/Concrete-Column-Crack-Detection-6")
if not DATA_DIR.exists():
    print("loading Roboflow workspace...")
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    print("loading Roboflow project...")
    project = rf.workspace(WORKSPACE).project(PROJECT)
    dataset = project.version(VERSION).download(EXPORT)
    DATA_DIR = Path(dataset.location)
else:
    print("Dataset sudah ada, skip download.")
print("Dataset location:", DATA_DIR)

# =====================[ 3) SANITASI LABEL: HAPUS 'Normal' ]=====
def polygon_to_bbox(coords):
    xs = coords[0::2]; ys = coords[1::2]
    if not xs or not ys: return None
    xmin, xmax = max(0.0, min(xs)), min(1.0, max(xs))
    ymin, ymax = max(0.0, min(ys)), min(1.0, max(ys))
    w = max(0.0, xmax - xmin); h = max(0.0, ymax - ymin)
    if w <= 0 or h <= 0: return None
    xc, yc = (xmin + xmax) / 2.0, (ymin + ymax) / 2.0
    return [xc, yc, w, h]

yaml_path = DATA_DIR / "data.yaml"
with open(yaml_path, "r") as f:
    data_cfg = yaml.safe_load(f)

names = list(data_cfg.get("names", []))
assert isinstance(names, list) and len(names) > 0, "names[] tidak ditemukan di data.yaml"

if "Normal" in names:
    NORMAL_IDX = names.index("Normal")
    names.pop(NORMAL_IDX)
    data_cfg["names"] = names
    data_cfg["nc"] = len(names)
    with open(yaml_path, "w") as f:
        yaml.safe_dump(data_cfg, f, sort_keys=False)
    print("Kelas setelah remove 'Normal':", data_cfg["names"])
    print("nc:", data_cfg["nc"])
    print("Index 'Normal' semula:", NORMAL_IDX)
else:
    NORMAL_IDX = None
    print("PERINGATAN: 'Normal' tidak ada di names[]. Names:", names)

def process_labels(lbl_dir, normal_idx):
    if not lbl_dir.exists(): return (0,0,0)
    n_files = 0; removed_files = 0; changed_lines = 0
    for p in lbl_dir.glob("*.txt"):
        n_files += 1
        txt = p.read_text().strip()
        if not txt:
            p.unlink(missing_ok=True); removed_files += 1; continue
        lines = [L.strip() for L in txt.splitlines() if L.strip()]
        new_lines = []
        for L in lines:
            parts = L.split()
            if len(parts) < 2: continue
            try:
                c = int(float(parts[0]))
            except:
                continue
            # 5 kolom bbox atau polygon → bbox
            bbox = None
            if len(parts) == 5:
                try: bbox = list(map(float, parts[1:5]))
                except: bbox = None
            elif len(parts) > 5:
                try:
                    coords = list(map(float, parts[1:]))
                    bbox = polygon_to_bbox(coords)
                except: bbox = None
            if bbox is None: continue

            if normal_idx is not None:
                if c == normal_idx:
                    changed_lines += 1; continue
                if c > normal_idx:
                    c -= 1; changed_lines += 1

            xc, yc, w, h = bbox
            xc = min(max(xc, 0.0), 1.0); yc = min(max(yc, 0.0), 1.0)
            w  = min(max(w,  0.0), 1.0); h  = min(max(h,  0.0), 1.0)
            if w == 0.0 or h == 0.0: continue
            new_lines.append(f"{c} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")

        if new_lines:
            p.write_text("\n".join(new_lines) + "\n")
        else:
            p.unlink(missing_ok=True); removed_files += 1
    return (n_files, changed_lines, removed_files)

for split in ["train", "valid", "val", "test"]:
    d = DATA_DIR / split / "labels"
    if d.exists():
        n, chg, rm = process_labels(d, NORMAL_IDX)
        print(f"Dir {d}: {n} files, {chg} baris diubah, {rm} file label dihapus.")

# bersihkan cache YOLO agar re-index
for cache in DATA_DIR.rglob("*.cache"):
    cache.unlink(missing_ok=True)

# =====================[ 4) (OPSIONAL) OVERSAMPLE RINGAN ]========
# Disarankan: True tapi konservatif agar tidak meledak jumlah data (hemat VRAM & I/O).
ENABLE_OVERSAMPLE = True
TARGET_MIN_PER_CLASS = 700     # target minimum per kelas
MAX_NEW_PAIRS = 600            # batas atas duplikat (global)

train_img = DATA_DIR / "train" / "images"
train_lbl = DATA_DIR / "train" / "labels"
train_img.mkdir(parents=True, exist_ok=True)
train_lbl.mkdir(parents=True, exist_ok=True)

def count_class_occurrences(lbl_dir, n_classes):
    counts = [0]*n_classes
    for p in lbl_dir.glob("*.txt"):
        for L in p.read_text().splitlines():
            if not L.strip(): continue
            f0 = L.split()[0]
            if f0.lstrip("-").isdigit():
                idx = int(float(f0))
                if 0 <= idx < n_classes:
                    counts[idx] += 1
    return counts

def oversample_minority(train_dir, target_min=700, max_new=600):
    img_dir = Path(train_dir)/"images"; lbl_dir = Path(train_dir)/"labels"
    marker = Path(train_dir)/".oversampled_light"
    if marker.exists():
        print("Skip oversample: sudah pernah dilakukan (marker ada).")
        return 0
    with open(yaml_path, "r") as f:
        cfg = yaml.safe_load(f)
    names_local = cfg["names"]; ncls = len(names_local)

    before = count_class_occurrences(lbl_dir, ncls)
    print("Class counts (train) sebelum oversample:")
    for i,c in enumerate(before):
        print(f"{i:2d} {names_local[i]:22s}: {c}")

    need = [max(0, target_min - c) for c in before]
    total_need = sum(need)
    if total_need == 0:
        marker.write_text("done"); print("Semua kelas >= target, skip oversample."); return 0

    added = 0
    for lbl in lbl_dir.glob("*.txt"):
        if added >= max_new: break
        lines = [L for L in lbl.read_text().splitlines() if L.strip()]
        if not lines: continue
        # cek kelas mana saja di file ini
        cls_in_file = set()
        for L in lines:
            f0 = L.split()[0]
            if f0.lstrip("-").isdigit():
                cls_in_file.add(int(float(f0)))
        # hanya gandakan kalau ada kelas yang masih kurang
        if not any(need[c] > 0 for c in cls_in_file):
            continue

        stem = lbl.stem
        img_path = None
        for ext in [".jpg",".jpeg",".png",".bmp",".JPG",".JPEG",".PNG"]:
            p = img_dir/f"{stem}{ext}"
            if p.exists(): img_path = p; break
        if img_path is None: continue

        new_img = img_dir / f"{stem}_dup{added}{img_path.suffix}"
        new_lbl = lbl_dir / f"{stem}_dup{added}.txt"
        if not new_img.exists():
            shutil.copy2(img_path, new_img)
            shutil.copy2(lbl, new_lbl)
            added += 1
            for c in cls_in_file:
                if need[c] > 0: need[c] -= 1
            if added >= max_new: break

    marker.write_text("done")
    after = count_class_occurrences(lbl_dir, ncls)
    print(f"Auto-oversample: ditambahkan {added} pasangan image/label.")
    print("Class counts (train) sesudah oversample:")
    for i,c in enumerate(after):
        print(f"{i:2d} {names_local[i]:22s}: {c}")
    return added

if ENABLE_OVERSAMPLE:
    # bersihkan duplikat lama (jika ada)
    removed_img = removed_lbl = 0
    for p in train_img.glob("*_dup*"): p.unlink(missing_ok=True); removed_img += 1
    for p in train_lbl.glob("*_dup*.txt"): p.unlink(missing_ok=True); removed_lbl += 1
    if removed_img or removed_lbl:
        print(f"Hapus duplikat lama: {removed_img} image, {removed_lbl} label")
    _ = oversample_minority(DATA_DIR / "train", TARGET_MIN_PER_CLASS, MAX_NEW_PAIRS)

# =====================[ 5) TRAINING (SAFE VRAM) ]================
from ultralytics import YOLO

# Konfigurasi aman T4 16GB
MODEL_WEIGHTS = "yolo12n.pt"     # model kecil → batch tetap lumayan
RUN_NAME = "crack_v12n_832_bal_safe"
EPOCHS   = 100
IMGSZ    = 832
BATCH    = 8                      # sesuaikan jika masih mepet: turunkan ke 4
NBS      = 8                      # harus sama dengan BATCH (tanpa accumulate)
PATIENCE = 40
WORKERS  = 2

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("\n=== CONFIG TRAIN (SAFE) ===")
print(f"Run Name : {RUN_NAME}")
print(f"Epochs   : {EPOCHS}, Patience: {PATIENCE}")
print(f"Image sz : {IMGSZ}")
print(f"Batch    : {BATCH}, nbs={NBS}  (tanpa accumulate)")
print("rect=True, cache='disk', multi_scale=False")

model = YOLO(MODEL_WEIGHTS)

results = model.train(
    data=str(yaml_path),
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    nbs=NBS,                   # ganti peran accumulate → jaga skala LR
    optimizer="AdamW",
    lr0=0.002, lrf=0.10,
    weight_decay=0.01,
    cos_lr=True,

    # augment ringan (ramah compute & retak)
    mosaic=0.05, close_mosaic=10,
    hsv_h=0.005, hsv_s=0.30, hsv_v=0.20,
    translate=0.05, scale=0.40,
    fliplr=0.5, flipud=0.0, mixup=0.0,
    auto_augment=None,
    erasing=0.0,

    # hemat mem & deterministik
    multi_scale=False,
    rect=True,
    cache='disk',
    workers=WORKERS,

    # loss weights konservatif
    iou=0.70, box=7.5, cls=0.5, dfl=1.5,

    patience=PATIENCE,
    device=0 if torch.cuda.is_available() else None,
    seed=0, deterministic=True,
    save=True, save_period=0,    # hanya best/last
    name=RUN_NAME,
    verbose=True
)

print("\nTraining selesai. Best dir:", results.save_dir)

# =====================[ 6) VALIDATE RINGKAS ]===================
metrics = model.val(data=str(yaml_path), imgsz=IMGSZ, plots=False)
print("mAP50    :", getattr(metrics.box, "map50", None))
print("mAP50-95 :", getattr(metrics.box, "map", None))

# =====================[ 7) (OPSIONAL) FINETUNE HI-RES ]=========
DO_HIGHRES_FINETUNE = True   # set False kalau mau skip
if DO_HIGHRES_FINETUNE:
    from pathlib import Path as _P

    best_pt = _P(str(results.save_dir)) / "weights" / "best.pt"
    if not best_pt.exists():
        best_pt = _P(str(results.save_dir)) / "weights" / "last.pt"

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    model_ft = YOLO(str(best_pt))
    RUN_NAME_FT = RUN_NAME + "_ft1024"
    results_ft = model_ft.train(
        data=str(yaml_path),
        epochs=10,
        imgsz=1024,
        batch=4,                # kecil agar aman
        nbs=4,                  # jaga skala LR
        optimizer="AdamW",
        lr0=0.0015, lrf=0.1,
        weight_decay=0.01,
        cos_lr=True,

        mosaic=0.05, close_mosaic=5,
        auto_augment=None, erasing=0.0,

        multi_scale=False,
        rect=True,
        cache='disk',
        workers=WORKERS,

        iou=0.70, box=7.5, cls=0.5, dfl=1.5,
        device=0 if torch.cuda.is_available() else None,
        patience=10,
        seed=0, deterministic=True,
        save=True, save_period=0,
        name=RUN_NAME_FT,
        verbose=True
    )
    print("Hi-res fine-tune selesai. Best dir:", results_ft.save_dir)
    metrics_ft = model_ft.val(data=str(yaml_path), imgsz=1024, plots=False)
    print("FT mAP50    :", getattr(metrics_ft.box, "map50", None))
    print("FT mAP50-95 :", getattr(metrics_ft.box, "map", None))

# =====================[ 8) EXPORT (opsional) ]==================
try:
    from pathlib import Path as _P
    save_dir = _P(str(results.save_dir))
    best_pt = save_dir / "weights" / "best.pt"
    if best_pt.exists():
        print("Best weights:", best_pt)
        YOLO(str(best_pt)).export(format="onnx", imgsz=IMGSZ, nms=True)
except Exception as e:
    print("Export ONNX gagal/skip:", e)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 182.2 MB/s eta 0:00:00
Python  : 3.12.11
PyTorch : 2.8.0+cu126 | CUDA: 12.6 | GPU: Tesla T4
NumPy   : 2.0.2
OpenCV  : 4.10.0
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Concrete-Column-Crack-Detection-6 in yolov12:: 100%|██████████| 12552/12552 [00:02<00:00, 4211.97it/s]


Dataset location: /content/Concrete-Column-Crack-Detection-6
Kelas setelah remove 'Normal': ['Corrosion_Minor', 'Corrosion_Moderate', 'Corrosion_Severe', 'Crack_Minor', 'Crack_Moderate', 'Crack_Severe', 'Honeycomb_Minor', 'Honeycomb_Moderate', 'Honeycomb_Severe', 'Segregation_Minor', 'Segregation_Moderate', 'Segregation_Severe', 'Spalling_Minor', 'Spalling_Moderate', 'Spalling_Severe']
nc: 15
Index 'Normal' semula: 9
Dir /content/Concrete-Column-Crack-Detection-6/train/labels: 5787 files, 5558 baris diubah, 93 file label dihapus.
Dir /content/Concrete-Column-Crack-Detection-6/valid/labels: 258 files, 221 baris diubah, 5 file label dihapus.
Dir /content/Concrete-Column-Crack-Detection-6/test/labels: 225 files, 179 baris diubah, 2 file label dihapus.
Class counts (train) sebelum oversample:
 0 Corrosion_Minor       : 5517
 1 Corrosion_Moderate    : 1671
 2 Corrosion_Severe      : 429
 3 Crack_Minor           : 1212
 4 Crack_Moderate        : 918
 5 Crack_Severe          : 117
 6 Honeycom